In [1]:
import pandas as pd
import numpy as np
import interface as itf

In [2]:
df = pd.read_csv('../dataset/clean/day_data_fixed.csv', header=0)

In [4]:
ex_df = itf.get_exercise(1)

In [5]:
print(ex_df.columns)

Index(['logId', 'activityName', 'activityTypeId', 'activityLevel',
       'averageHeartRate', 'calories', 'activeDuration', 'steps', 'startTime',
       'elevationGain'],
      dtype='object')


In [6]:
ex_df['exerciseDate'] = pd.to_datetime(pd.to_datetime(ex_df['startTime']).dt.date)

In [7]:
ex_df.columns

Index(['logId', 'activityName', 'activityTypeId', 'activityLevel',
       'averageHeartRate', 'calories', 'activeDuration', 'steps', 'startTime',
       'elevationGain', 'exerciseDate'],
      dtype='object')

In [10]:
ex_df = ex_df.drop(columns=['logId', 'activityTypeId', 'activityLevel', 'startTime'])

In [25]:
agg_df = ex_df.groupby('exerciseDate').agg(
    {
        'activityName': set,
        'averageHeartRate': 'mean',
        'calories': 'sum',
        'activeDuration': 'sum',
        'steps': 'sum',
        'elevationGain': 'sum'
    }
).reset_index()
agg_df['activeDuration'] /= (1000*60)

In [26]:
out_df = agg_df.rename(columns={'steps': 'active_steps', 'averageHeartRate':'active_average_hr', 'calories': 'active_calories', 'exerciseDate':'dateTime'})

In [27]:
out_df.columns

Index(['dateTime', 'activityName', 'active_average_hr', 'active_calories',
       'activeDuration', 'active_steps', 'elevationGain'],
      dtype='object')

In [40]:
old_df = pd.read_csv('../dataset/clean/day_data_v2.csv', header=0)

In [41]:
old_df['dateTime'] = pd.to_datetime(old_df['dateTime'])
final_df = old_df.merge(out_df, on='dateTime', how='left')

In [43]:
print(final_df[final_df.isna().any(axis=1)])
print(out_df)

       dateTime  laminutes  maminutes  vaminutes  sedminutes     resthr  \
1    2019-11-03      262.0         63         45         648  53.222024   
4    2019-11-06      206.0         10         11         849  54.203189   
5    2019-11-07      182.0         11         15         866  53.090003   
6    2019-11-07      182.0         11         15         866  53.090003   
12   2019-11-13      197.0         24         33         825  51.889327   
...         ...        ...        ...        ...         ...        ...   
1660 2020-03-20       62.0          9          5         774  68.000998   
1662 2020-03-22       60.0          0          0         825  66.569772   
1663 2020-03-23       85.0          8          1         890  68.090240   
1664 2020-03-24       84.0          0          0         809  67.953362   
1670 2020-03-30       69.0          9          4         814  65.947171   

           sleepStartTime             sleepEndTime  totalSleepDuration  \
1     2019-11-03 00:32:00